In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.transform import xy
import numpy as np

Ahn-4 data inlezen en omzetten in pandas dataframe met coördinaten 

In [2]:
with rasterio.open("B:\AHN4\DTM_5m\Totaal\GEOHGT_AHN004__5MTRDTMRST.tif") as ahn_data:
    image = ahn_data.read(1) 
    transform = ahn_data.transform

rows, cols = image.shape
row_indices, col_indices = np.meshgrid(np.arange(rows), np.arange(cols), indexing="ij")

longitudes, latitudes = xy(transform, row_indices, col_indices)

ahn_df = pd.DataFrame({
    "longitude": longitudes.flatten(),
    "latitude": latitudes.flatten(),
    "hoogte_ahn": image.flatten()
})

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\tdubb\AppData\Local\Temp\ipykernel_22852\448361795.py:1: SyntaxWarning: invalid escape sequence '\A'
  with rasterio.open("B:\AHN4\DTM_5m\Totaal\GEOHGT_AHN004__5MTRDTMRST.tif") as ahn_data:


In [3]:
with rasterio.open("B:\AHN4\DTM_5m\Totaal\GEOHGT_AHN004__5MTRDTMRST.tif") as image:
    crs = image.crs

print(crs)

EPSG:28992


<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\tdubb\AppData\Local\Temp\ipykernel_22852\3138606640.py:1: SyntaxWarning: invalid escape sequence '\A'
  with rasterio.open("B:\AHN4\DTM_5m\Totaal\GEOHGT_AHN004__5MTRDTMRST.tif") as image:


Lege punten opschonen 

In [4]:
nodata_value = ahn_df["hoogte_ahn"].loc[ahn_df["hoogte_ahn"] < 10000].min()

ahn_df["hoogte_ahn"] = ahn_df["hoogte_ahn"].replace(nodata_value, np.nan)
ahn_df_cleaned = ahn_df.dropna(subset=["hoogte_ahn"])

ahn_df_cleaned

,longitude,latitude,hoogte_ahn
7837,49187.5,424997.5,-1.023828
7838,49192.5,424997.5,-0.864958
7839,49197.5,424997.5,-0.722748
7840,49202.5,424997.5,-0.589853
7841,49207.5,424997.5,-0.457312
...,...,...,...
188070019,50097.5,357832.5,1.704777
188070020,50102.5,357832.5,1.776526
188070021,50107.5,357832.5,1.868386
188084020,50102.5,357827.5,1.735669


Merg een sample: van Ahn4 data en data uit chloride_klassen_csv 

In [5]:
ahn_data = ahn_df_cleaned
chloride_klassen_df = pd.read_csv("A:/FreshEM/Zeeland/3D/csv/chloride_klassen.csv")
#chloride_klassen_df = pd.read_csv(r"A:\FreshEM\Zeeland\2D\Midden_Zeeland\csv\FRESHEM Midden Zeeland 2D resultaat vlieglijnen.csv")
#chloride_klassen_df = pd.read_excel(r"C:\Users\tdubb\Downloads\test.xlsx")

In [6]:
chloride_klassen_df

,SN+ Sample Number (READONLY)+,XG+ Gravity Center+,YG+ Gravity Center+,ZG+ Gravity Center+,chloride_klassen{laag},chloride_klassen{midden},chloride_klassen{hoog}
0,2527852,13775.0,370825.0,-40.25,0,0,0
1,2527853,13775.0,370825.0,-39.75,0,0,0
2,2527854,13775.0,370825.0,-39.25,0,0,0
3,2527855,13775.0,370825.0,-38.75,0,0,0
4,2527856,13775.0,370825.0,-38.25,0,0,0
...,...,...,...,...,...,...,...
52586237,627467439,78225.0,379925.0,-1.75,0,0,0
52586238,627467440,78225.0,379925.0,-1.25,0,0,0
52586239,627467441,78225.0,379925.0,-0.75,0,0,500
52586240,627467442,78225.0,379925.0,-0.25,0,0,2000


In [7]:
# Sorteer datasets
chloride_klassen_sample = chloride_klassen_df.sort_values(by="ZG+ Gravity Center+")
ahn_sample =ahn_data.sort_values(by="longitude")

# Print aantal rijen voor filtering
print("Aantal rijen vóór filtering:", len(chloride_klassen_sample))

# Filter eerst op ZG+ Gravity Center+	
chloride_klassen_sample_filtered = chloride_klassen_sample[
    chloride_klassen_sample["ZG+ Gravity Center+"] >= -5
]

# Print aantal rijen na filtering
print("Aantal rijen ná filtering:", len(chloride_klassen_sample_filtered))

Aantal rijen vóór filtering: 52586242
Aantal rijen ná filtering: 7430146


GEO dataframes (coördianten als points)

In [8]:
ahn_sample_gdf = gpd.GeoDataFrame(
    ahn_sample, 
    geometry=gpd.points_from_xy(ahn_sample["longitude"], ahn_sample["latitude"]),
    crs="EPSG:28992"
)

#Meters omzetten in centimers
ahn_sample_gdf["hoogte_ahn"] = ahn_sample_gdf["hoogte_ahn"] * 100
chloride_klassen_sample_filtered["ZG+ Gravity Center+"] = chloride_klassen_sample_filtered["ZG+ Gravity Center+"] * 100


#Corrigeren voor de extra 00 op het einde van elk coordinaat
#chloride_klassen_sample_filtered["XG+ Gravity Center+"] = chloride_klassen_sample_filtered["XG+ Gravity Center+"] / 100
#chloride_klassen_sample_filtered["YG+ Gravity Center+"] = chloride_klassen_sample_filtered["YG+ Gravity Center+"] / 100

chloride_klassen_sample_gdf = gpd.GeoDataFrame(
    chloride_klassen_sample_filtered,
    geometry=gpd.points_from_xy(
        chloride_klassen_sample_filtered["XG+ Gravity Center+"],
        chloride_klassen_sample_filtered["YG+ Gravity Center+"]
    ),
    crs="EPSG:28992"
)

C:\Users\tdubb\AppData\Local\Temp\ipykernel_22852\3479352122.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chloride_klassen_sample_filtered["ZG+ Gravity Center+"] = chloride_klassen_sample_filtered["ZG+ Gravity Center+"] * 100


In [9]:
chloride_klassen_sample_gdf

,SN+ Sample Number (READONLY)+,XG+ Gravity Center+,YG+ Gravity Center+,ZG+ Gravity Center+,chloride_klassen{laag},chloride_klassen{midden},chloride_klassen{hoog},geometry
8475916,134546823,27375.0,391525.0,-475.0,10000,15000,15000,POINT (27375 391525)
27338304,335732223,48125.0,392275.0,-475.0,7500,10000,15000,POINT (48125 392275)
1055681,33044253,16925.0,367725.0,-475.0,0,0,0,POINT (16925 367725)
3712516,77395833,21475.0,398175.0,-475.0,15000,15000,15000,POINT (21475 398175)
17914428,256251393,39925.0,395025.0,-475.0,15000,15000,15000,POINT (39925 395025)
...,...,...,...,...,...,...,...,...
5547010,99641096,23775.0,391225.0,2175.0,0,0,0,POINT (23775 391225)
5592024,100126256,23825.0,391275.0,2175.0,0,0,0,POINT (23825 391275)
5592025,100126257,23825.0,391275.0,2225.0,0,0,0,POINT (23825 391275)
5547011,99641097,23775.0,391225.0,2225.0,0,0,0,POINT (23775 391225)


Nearest-neighbor join

In [10]:
merged_gdf = gpd.sjoin_nearest(
    chloride_klassen_sample_gdf, ahn_sample_gdf,
    how="left", 
    distance_col="distance",
    exclusive=True
)

In [11]:
#Duplicaten verwijderen
ahn_mean = (
    merged_gdf.groupby(
        ["XG+ Gravity Center+", "YG+ Gravity Center+", "ZG+ Gravity Center+"],
        as_index=False
    )
    .agg({"hoogte_ahn": "mean"})
)

# Stap 2: Verwijder duplicaten op basis van XYZ
merged_dedup = merged_gdf.drop_duplicates(
    subset=["XG+ Gravity Center+", "YG+ Gravity Center+", "ZG+ Gravity Center+"],
    keep="first"
).copy()

# Stap 3: Vervang hoogte_ahn door het groepsgemiddelde
merged_dedup = merged_dedup.drop(columns=["hoogte_ahn"])
merged_dedup = merged_dedup.merge(
    ahn_mean,
    on=["XG+ Gravity Center+", "YG+ Gravity Center+", "ZG+ Gravity Center+"],
    how="left"
)

In [12]:
merged_dedup

,SN+ Sample Number (READONLY)+,XG+ Gravity Center+,YG+ Gravity Center+,ZG+ Gravity Center+,chloride_klassen{laag},chloride_klassen{midden},chloride_klassen{hoog},geometry,index_right,longitude,latitude,distance,hoogte_ahn
0,134546823,27375.0,391525.0,-475.0,10000,15000,15000,POINT (27375 391525),93733475,27377.5,391522.5,3.535534,-62.689102
1,335732223,48125.0,392275.0,-475.0,7500,10000,15000,POINT (48125 392275),91637624,48122.5,392272.5,3.535534,-134.049042
2,33044253,16925.0,367725.0,-475.0,0,0,0,POINT (16925 367725),160371384,16922.5,367722.5,3.535534,80.678528
3,77395833,21475.0,398175.0,-475.0,15000,15000,15000,POINT (21475 398175),76050345,21727.5,397837.5,421.500297,143.971710
4,256251393,39925.0,395025.0,-475.0,15000,15000,15000,POINT (39925 395025),83921982,39912.5,395027.5,12.747549,-15.028227
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430141,99641096,23775.0,391225.0,2175.0,0,0,0,POINT (23775 391225),94558755,23777.5,391227.5,3.535534,3336.012207
7430142,100126256,23825.0,391275.0,2175.0,0,0,0,POINT (23825 391275),94432765,23827.5,391272.5,3.535534,2966.025879
7430143,100126257,23825.0,391275.0,2225.0,0,0,0,POINT (23825 391275),94432765,23827.5,391272.5,3.535534,2966.025879
7430144,99641097,23775.0,391225.0,2225.0,0,0,0,POINT (23775 391225),94558755,23777.5,391227.5,3.535534,3336.012207


In [13]:
merged_dedup["hoogte_tov_maaiveld"] = merged_dedup["ZG+ Gravity Center+"] - merged_dedup["hoogte_ahn"]


In [14]:
merged_dedup

,SN+ Sample Number (READONLY)+,XG+ Gravity Center+,YG+ Gravity Center+,ZG+ Gravity Center+,chloride_klassen{laag},chloride_klassen{midden},chloride_klassen{hoog},geometry,index_right,longitude,latitude,distance,hoogte_ahn,hoogte_tov_maaiveld
0,134546823,27375.0,391525.0,-475.0,10000,15000,15000,POINT (27375 391525),93733475,27377.5,391522.5,3.535534,-62.689102,-412.310898
1,335732223,48125.0,392275.0,-475.0,7500,10000,15000,POINT (48125 392275),91637624,48122.5,392272.5,3.535534,-134.049042,-340.950958
2,33044253,16925.0,367725.0,-475.0,0,0,0,POINT (16925 367725),160371384,16922.5,367722.5,3.535534,80.678528,-555.678528
3,77395833,21475.0,398175.0,-475.0,15000,15000,15000,POINT (21475 398175),76050345,21727.5,397837.5,421.500297,143.971710,-618.971710
4,256251393,39925.0,395025.0,-475.0,15000,15000,15000,POINT (39925 395025),83921982,39912.5,395027.5,12.747549,-15.028227,-459.971773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430141,99641096,23775.0,391225.0,2175.0,0,0,0,POINT (23775 391225),94558755,23777.5,391227.5,3.535534,3336.012207,-1161.012207
7430142,100126256,23825.0,391275.0,2175.0,0,0,0,POINT (23825 391275),94432765,23827.5,391272.5,3.535534,2966.025879,-791.025879
7430143,100126257,23825.0,391275.0,2225.0,0,0,0,POINT (23825 391275),94432765,23827.5,391272.5,3.535534,2966.025879,-741.025879
7430144,99641097,23775.0,391225.0,2225.0,0,0,0,POINT (23775 391225),94558755,23777.5,391227.5,3.535534,3336.012207,-1111.012207


In [15]:
import openpyxl
merged_dedup.to_csv("ahn4_en_chloride_klassen.csv", index=False)

# Klassen maken van de nieuwe maaiveld hoogtes

In [16]:
df = merged_dedup.copy()

# Alles boven 10 cm boven maaiveld negeren (optioneel)
df = df[df["hoogte_tov_maaiveld"] <= 10]

# Alles onder de -300 cm onder maaiveld negeren
df = df[df["hoogte_tov_maaiveld"] >= -300]

# Voor alles tussen -50 en 10: zet handmatig op klasse "0 to -50"
df["hoogteklasse"] = np.where(
    df["hoogte_tov_maaiveld"] > -50,
    "0 to -50",
    pd.cut(
        df["hoogte_tov_maaiveld"],
        bins=np.arange(-300, -49, 50),
        labels=[f"{a} to {a + 49}" for a in range(-300, -50, 50)],
        include_lowest=True
    ).astype(str)
)

df

,SN+ Sample Number (READONLY)+,XG+ Gravity Center+,YG+ Gravity Center+,ZG+ Gravity Center+,chloride_klassen{laag},chloride_klassen{midden},chloride_klassen{hoog},geometry,index_right,longitude,latitude,distance,hoogte_ahn,hoogte_tov_maaiveld,hoogteklasse
2379,368350653,51475.0,410075.0,-475.0,7500,10000,15000,POINT (51475 410075),41840294,51472.5,410057.5,17.677670,-179.856903,-295.143097,-300 to -251
2425,368351043,51475.0,410125.0,-475.0,7500,10000,15000,POINT (51475 410125),41742287,51437.5,410092.5,49.623583,-178.851425,-296.148575,-300 to -251
2618,368351433,51475.0,410175.0,-475.0,7500,10000,10000,POINT (51475 410175),41392286,51432.5,410217.5,60.104076,-175.939178,-299.060822,-300 to -251
2667,368351823,51475.0,410225.0,-475.0,7500,10000,10000,POINT (51475 410225),41294292,51462.5,410252.5,30.207615,-178.238037,-296.761963,-300 to -251
2840,368352213,51475.0,410275.0,-475.0,7500,10000,10000,POINT (51475 410275),41238297,51487.5,410272.5,12.747549,-178.415237,-296.584763,-300 to -251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7429996,107870863,24625.0,389775.0,1525.0,0,0,0,POINT (24625 389775),98632925,24627.5,389772.5,3.535534,1718.883423,-193.883423,-200 to -151
7430002,107870864,24625.0,389775.0,1575.0,0,0,0,POINT (24625 389775),98632925,24627.5,389772.5,3.535534,1718.883423,-143.883423,-150 to -101
7430039,107870865,24625.0,389775.0,1625.0,0,0,0,POINT (24625 389775),98632925,24627.5,389772.5,3.535534,1718.883423,-93.883423,-100 to -51
7430046,107870866,24625.0,389775.0,1675.0,0,0,0,POINT (24625 389775),98632925,24627.5,389772.5,3.535534,1718.883423,-43.883423,0 to -50


In [17]:
# Hernoem kolom 'chloride_klassen{midden}' naar 'chloride_midden'
df = df.rename(columns={'chloride_klassen{midden}': 'chloride_midden'})

# Voeg een kolom toe om aan te geven of een waarde gemeten was (1)
df['aantal_gemeten'] = 1

# Bereken het gemiddelde en het aantal per combinatie bestaande kolommen
grouped_df = df.groupby(['XG+ Gravity Center+', 'YG+ Gravity Center+', 'hoogteklasse', 'hoogte_tov_maaiveld', 'hoogte_ahn'], as_index=False).agg(
    gemiddelde_chloride=('chloride_midden', 'mean'),
    aantal_gemeten=('aantal_gemeten', 'sum')
)

# Vind alle unieke combinaties van bestaande x, y en hoogteklasse
unieke_combinaties = pd.MultiIndex.from_product(
    [df['XG+ Gravity Center+'].unique(), df['YG+ Gravity Center+'].unique(), df['hoogteklasse'].unique()],
    names=['XG+ Gravity Center+', 'YG+ Gravity Center+', 'hoogteklasse']
).to_frame(index=False)

# Voeg ontbrekende combinaties toe
ontbrekende_combinaties = unieke_combinaties.merge(grouped_df, on=['XG+ Gravity Center+', 'YG+ Gravity Center+', 'hoogteklasse'], how='left', indicator=True)
ontbrekende_combinaties = ontbrekende_combinaties[ontbrekende_combinaties['_merge'] == 'left_only'].drop(columns=['gemiddelde_chloride', 'aantal_gemeten', 'hoogte_tov_maaiveld', 'hoogte_ahn', '_merge'])

# Voeg ontbrekende combinaties toe aan resultaat
ontbrekende_combinaties['gemiddelde_chloride'] = 0
ontbrekende_combinaties['aantal_gemeten'] = 0

# Gegevens combineren
resultaat_df = pd.concat([grouped_df, ontbrekende_combinaties], ignore_index=True)

# Voeg geometrie opnieuw toe op basis van bestaande x, y kolommen
resultaat_df['geometry'] = resultaat_df.apply(lambda row: Point(row['XG+ Gravity Center+'], row['YG+ Gravity Center+']), axis=1)
resultaat_gdf = gpd.GeoDataFrame(resultaat_df, geometry='geometry')

# Toon resultaat
resultaat_gdf

,XG+ Gravity Center+,YG+ Gravity Center+,hoogteklasse,hoogte_tov_maaiveld,hoogte_ahn,gemiddelde_chloride,aantal_gemeten,geometry
0,13775.0,370825.0,-100 to -51,-83.980759,58.980759,0.0,1,POINT (13775 370825)
1,13775.0,370825.0,-150 to -101,-133.980759,58.980759,0.0,1,POINT (13775 370825)
2,13775.0,370825.0,-200 to -151,-183.980759,58.980759,0.0,1,POINT (13775 370825)
3,13775.0,370825.0,-250 to -201,-233.980759,58.980759,0.0,1,POINT (13775 370825)
4,13775.0,370825.0,-300 to -251,-283.980759,58.980759,0.0,1,POINT (13775 370825)
...,...,...,...,...,...,...,...,...
9533423,78225.0,361425.0,-250 to -201,NaN,NaN,0.0,0,POINT (78225 361425)
9533424,78225.0,361425.0,-150 to -101,NaN,NaN,0.0,0,POINT (78225 361425)
9533425,78225.0,361425.0,-200 to -151,NaN,NaN,0.0,0,POINT (78225 361425)
9533426,78225.0,361425.0,-100 to -51,NaN,NaN,0.0,0,POINT (78225 361425)


In [18]:
resultaat_df.to_csv("ahn4_en_chloride_hoogteklassen.csv", index=False)

# Exporteren naar tiff files

In [22]:
import os
import rasterio
from rasterio.transform import from_origin


In [ ]:
# === Instelbare outputmap ===
output_folder = r"C:\Users\tdubb\Downloads\tiffs"

# Zorg dat de map bestaat
os.makedirs(output_folder, exist_ok=True)

df = resultaat_df

# Inputdata: df met kolommen: X, Y, chloride_midden, hoogteklasse
df = df.copy()  # of: df = grouped_gdf.copy() als het nog een GeoDataFrame is

# Zorg dat kolomnamen correct zijn
df = df.rename(columns={
    "XG+ Gravity Center+": "X",
    "YG+ Gravity Center+": "Y"
})

# Rasterinstellingen
resolution = 50  # pas aan indien nodig

# Verwerk per hoogteklasse
for klasse in df["hoogteklasse"].unique():
    subset = df[df["hoogteklasse"] == klasse]

    if subset.empty:
        continue

    # Raster extent
    xmin, ymin = subset["X"].min(), subset["Y"].min()
    xmax, ymax = subset["X"].max(), subset["Y"].max()

    width = int(np.ceil((xmax - xmin) / resolution)) + 1
    height = int(np.ceil((ymax - ymin) / resolution)) + 1

    # Initialiseer raster met NaN
    raster = np.full((height, width), np.nan)

    # Vul raster
    for _, row in subset.iterrows():
        col = int((row["X"] - xmin) / resolution)
        row_idx = int((ymax - row["Y"]) / resolution)
        raster[row_idx, col] = row["gemiddelde_chloride"]

    # Definieer affine transform
    transform = from_origin(xmin, ymax, resolution, resolution)

    # Outputpad
    klasse_label = klasse.replace(" ", "").replace("to", "tot")
    out_path = os.path.join(output_folder, f"chloride_{klasse_label}.tif")

    # Schrijf raster
    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype="float32",
        crs="EPSG:28992",
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(raster.astype("float32"), 1)

    print(f"Raster opgeslagen: {out_path}")
